In [1]:
import pandas as pd
import dask.dataframe as dd
import os

In [ ]:
#remove the header from the csv files; only need to do this once
#directory = 'E:/Data/SnowData/Training/'
#for filename in os.listdir(directory):
#    if filename.endswith('.csv'): 
#        f = open(os.path.join(directory, filename), 'r')
#        f2 = open(os.path.join('E:/Data/SnowData/TrainingCleaned/', filename), 'w')
#        header = True
#        for line in f:
#            if(line.startswith('#')):
#                continue
#            else:
#                if(header):
#                    f2.write(line.strip() + ",State\n")
#                    header = False
#                else:
#                    f2.write(line.strip() + "," + filename[6:8] + "\n")
#        f.close
#        f2.close
#    else:
#        continue


In [29]:
#read all states for a season
seasonDf = dd.read_csv('E:/Data/SnowData/TrainingCleaned/snotel*1617.csv', assume_missing=True, parse_dates=[0])
#TODO: need to make sure all data is parsed correctly includign the dates


In [30]:
seasonDf.head()

,Date,Station Name,Elevation (ft),Latitude,Longitude,Snow Water Equivalent (in),Precipitation Accumulation (in),Snow Depth (in),Air Temperature Observed (degF),State
0,2016-11-01 00:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.0,0.5,0.0,35.0,AK
1,2016-11-01 01:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,35.0,AK
2,2016-11-01 02:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,34.0,AK
3,2016-11-01 03:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,33.0,AK
4,2016-11-01 04:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,32.0,AK


In [31]:
#add 8 hours (do not adjust for DST) and correct date output format
seasonDf['Date'] +=  pd.to_timedelta(8, unit='h')

In [32]:
seasonDf.head()

,Date,Station Name,Elevation (ft),Latitude,Longitude,Snow Water Equivalent (in),Precipitation Accumulation (in),Snow Depth (in),Air Temperature Observed (degF),State
0,2016-11-01 08:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.0,0.5,0.0,35.0,AK
1,2016-11-01 09:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,35.0,AK
2,2016-11-01 10:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,34.0,AK
3,2016-11-01 11:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,33.0,AK
4,2016-11-01 12:00:00,Alexander Lake,160.0,61.74967,-150.88967,0.3,0.5,0.0,32.0,AK


In [33]:
#pull frame by hour
dates = seasonDf['Date'].unique().compute()

In [34]:
dates.head()

0   2016-11-01 08:00:00
1   2016-11-01 09:00:00
2   2016-11-01 10:00:00
3   2016-11-01 11:00:00
4   2016-11-01 12:00:00
Name: Date, dtype: datetime64[ns]

In [35]:
seasonComputed = seasonDf.compute()

In [36]:
for d in dates:
    hourDf = seasonComputed[seasonComputed['Date']==d]
    hourDf.to_csv('E:/Data/SnowData/TrainingParsed/16-17/' + d.strftime('%Y%m%d.%H') + '.csv', date_format="%Y%m%d %H:00", index=False)

In [ ]:
hourDf.head()

In [53]:
from shutil import copy
#put them in the correct directory layout
directory = 'E:/Data/SnowData/TrainingParsed/17-18/'
destination = 'E:/Data/SnowData/TrainingOrganized/'
for filename in os.listdir(directory):
    if filename.endswith('.csv'): 
        year = filename[:4]
        month = filename[4:6]
        day = filename[6:8]
        complete_destination = os.path.join(destination, year + '/' + month + '/' + day + '/')
        if(not os.path.isdir(complete_destination)):
            os.makedirs(complete_destination,mode=0o777)
            
        copy(os.path.join(directory, filename), complete_destination)
        